# Projection Results Dashboard

This notebook creates interactive visualizations of headcount trends from the projection results.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import os

# Set up styling
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")

# Define the results directory
results_dir = Path('../output_dev/projection_cli_results')

## Load Employment Status Summary

First, let's load the employment status summary which contains headcount by category per year.

In [ ]:
# Load the employment status summary
emp_status_path = results_dir / 'projection_cli_employment_status_summary.parquet'
emp_status_df = pd.read_parquet(emp_status_path)

# Display the data
print(f"Employment status data shape: {emp_status_df.shape}")
emp_status_df

## Load Yearly Snapshots

Let's also load the yearly snapshots to get more detailed information.

In [ ]:
# Load yearly snapshots
snapshots_dir = results_dir / 'yearly_snapshots'
yearly_snapshots = {}
yearly_counts = []

for snapshot_file in snapshots_dir.glob('*.parquet'):
    year = int(snapshot_file.stem.split('_')[-1])  # Extract year from filename
    df = pd.read_parquet(snapshot_file)
    yearly_snapshots[year] = df
    
    # Count active employees
    active_count = df[df.get('active', False) == True].shape[0]
    
    # Count by tenure band if available
    tenure_counts = {}
    if 'tenure_band' in df.columns:
        active_df = df[df.get('active', False) == True]
        tenure_counts = active_df['tenure_band'].value_counts().to_dict()
    
    yearly_counts.append({
        'Year': year,
        'Active Headcount': active_count,
        **{f"Tenure {band}": count for band, count in tenure_counts.items()}
    })

# Create a DataFrame with yearly counts
yearly_counts_df = pd.DataFrame(yearly_counts).sort_values('Year')
yearly_counts_df

## Interactive Headcount Plot

Now let's create an interactive plot of headcount vs. year using Plotly.

In [ ]:
# Create an interactive plot of headcount vs. year
if not emp_status_df.empty:
    # Prepare data for plotting
    plot_data = emp_status_df[['Year', 'Active', 'Continuous Active', 'New Hire Active', 
                              'Experienced Terminated', 'New Hire Terminated']].copy()
    
    # Create the interactive plot
    fig = px.line(plot_data, x='Year', y=['Active', 'Continuous Active', 'New Hire Active', 
                                         'Experienced Terminated', 'New Hire Terminated'],
                 title='Headcount Trends by Employment Status',
                 markers=True,
                 labels={'value': 'Count', 'variable': 'Status'},
                 template='plotly_white')
    
    # Add data points with hover information
    for column in ['Active', 'Continuous Active', 'New Hire Active', 'Experienced Terminated', 'New Hire Terminated']:
        fig.add_trace(
            go.Scatter(
                x=plot_data['Year'],
                y=plot_data[column],
                mode='markers',
                name=f'{column} (points)',
                hovertemplate=f'{column}: %{{y}}<br>Year: %{{x}}<extra></extra>',
                showlegend=False
            )
        )
    
    # Update layout
    fig.update_layout(
        xaxis=dict(tickmode='linear', dtick=1),
        hovermode='closest',
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
    )
    
    fig.show()
else:
    # Use the yearly_counts_df if employment status data is not available
    fig = px.line(yearly_counts_df, x='Year', y='Active Headcount',
                 title='Active Headcount by Year',
                 markers=True,
                 labels={'Active Headcount': 'Count'},
                 template='plotly_white')
    
    fig.update_layout(
        xaxis=dict(tickmode='linear', dtick=1),
        hovermode='closest'
    )
    
    fig.show()

## Stacked Area Chart of Employment Status

Let's create a stacked area chart to visualize the composition of the workforce over time.

In [ ]:
if not emp_status_df.empty:
    # Create a stacked area chart
    fig = px.area(plot_data, x='Year', 
                 y=['Continuous Active', 'New Hire Active', 'Experienced Terminated', 'New Hire Terminated'],
                 title='Workforce Composition Over Time',
                 labels={'value': 'Count', 'variable': 'Status'},
                 template='plotly_white')
    
    fig.update_layout(
        xaxis=dict(tickmode='linear', dtick=1),
        hovermode='x unified',
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
    )
    
    fig.show()

## Tenure Band Analysis

If tenure band data is available, let's visualize the distribution of employees by tenure.

In [ ]:
# Check if we have tenure band data
tenure_columns = [col for col in yearly_counts_df.columns if col.startswith('Tenure ')]

if tenure_columns:
    # Create a stacked bar chart of tenure bands
    fig = px.bar(yearly_counts_df, x='Year', y=tenure_columns,
                title='Active Employees by Tenure Band',
                labels={'value': 'Count', 'variable': 'Tenure Band'},
                template='plotly_white')
    
    fig.update_layout(
        xaxis=dict(tickmode='linear', dtick=1),
        hovermode='x unified',
        barmode='stack',
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
    )
    
    fig.show()

## Summary Statistics

Let's also look at the overall summary statistics.

In [ ]:
# Load summary statistics
stats_path = results_dir / 'projection_cli_summary_statistics.parquet'
if stats_path.exists():
    stats_df = pd.read_parquet(stats_path)
    print(f"Summary statistics data shape: {stats_df.shape}")
    stats_df

In [ ]:
# If we have summary statistics, create visualizations for key metrics
if 'stats_df' in locals() and not stats_df.empty:
    # Select numeric columns that might be interesting to plot
    numeric_cols = stats_df.select_dtypes(include=['number']).columns.tolist()
    year_col = 'Year' if 'Year' in stats_df.columns else stats_df.index.name
    
    # Create subplots for key metrics
    metrics_to_plot = [col for col in numeric_cols if col != year_col and 'Headcount' in col or 'Count' in col or 'Rate' in col]
    
    if metrics_to_plot:
        fig = px.line(stats_df, x=year_col, y=metrics_to_plot[:5],  # Limit to 5 metrics for readability
                     title='Key Metrics Over Time',
                     markers=True,
                     labels={'value': 'Value', 'variable': 'Metric'},
                     template='plotly_white')
        
        fig.update_layout(
            xaxis=dict(tickmode='linear', dtick=1),
            hovermode='x unified',
            legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
        )
        
        fig.show()

## Experienced Terminated Analysis

Let's specifically look at the 'Experienced Terminated' category, which was the focus of our investigation.

In [ ]:
if not emp_status_df.empty and 'Experienced Terminated' in emp_status_df.columns:
    # Create a focused plot on Experienced Terminated
    fig = px.bar(emp_status_df, x='Year', y='Experienced Terminated',
                title='Experienced Terminated Employees by Year',
                labels={'Experienced Terminated': 'Count'},
                template='plotly_white')
    
    # Add a line for the percentage of experienced terminated relative to active
    emp_status_df['Experienced Terminated %'] = (emp_status_df['Experienced Terminated'] / 
                                               emp_status_df['Active']) * 100
    
    fig.add_trace(
        go.Scatter(
            x=emp_status_df['Year'],
            y=emp_status_df['Experienced Terminated %'],
            mode='lines+markers',
            name='% of Active',
            yaxis='y2',
            line=dict(color='red'),
            hovertemplate='%{y:.1f}% of Active<extra></extra>'
        )
    )
    
    fig.update_layout(
        xaxis=dict(tickmode='linear', dtick=1),
        yaxis2=dict(
            title='Percentage of Active',
            overlaying='y',
            side='right',
            range=[0, max(emp_status_df['Experienced Terminated %']) * 1.2]
        ),
        hovermode='x unified'
    )
    
    fig.show()